In [1]:
import pandas as pd
import numpy as np
import requests
import grequests
import os.path

In [2]:
# auxiliary function for below
def split(theList, n):
    for i in range(0, len(theList), n):
        yield theList[i:i + n]

In [3]:
host = "https://api.stratz.com/api/v1/match"

In [4]:
params = {"matchId":[3304258209,3304204784], 
          "include": ["Player"],
         "gameMode": [2, 22],
         "lobbyType": [2, 7]}

In [5]:
x = requests.get(host, params = params)

In [9]:
x

<Response [200]>

In [15]:
host = "https://api.stratz.com/api/v1/match/3304258209"
rs = (grequests.get(host, session=session),)
x = grequests.map(rs)
data = x[0].json()

In [16]:
keys = data.keys()
print(keys)

dict_keys(['avgImp', 'clusterId', 'firstBloodTime', 'rank', 'courierEvents', 'gameResult', 'gameMode', 'bracket', 'endDate', 'regionId', 'didRadiantWin', 'parsedDate', 'challenge', 'tier', 'roshanEvents', 'players', 'towerDeathEvents', 'radiantKills', 'lobbyType', 'replaySalt', 'buildingEvents', 'duration', 'id', 'direKills', 'gameVersionId', 'isStats', 'wardEvents', 'runeEvents'])


In [17]:
testsequence = [event['item'] for event in data['players'][0]['purchaseEvents']]
testitems = [45, 42, 44]
list(filter(lambda a: a not in testitems, testsequence))

[188,
 43,
 29,
 38,
 46,
 46,
 84,
 244,
 27,
 27,
 214,
 46,
 46,
 46,
 25,
 182,
 46,
 46,
 64,
 65,
 46,
 46,
 23,
 21,
 22,
 60,
 108]

In [18]:
keys

dict_keys(['avgImp', 'clusterId', 'firstBloodTime', 'rank', 'courierEvents', 'gameResult', 'gameMode', 'bracket', 'endDate', 'regionId', 'didRadiantWin', 'parsedDate', 'challenge', 'tier', 'roshanEvents', 'players', 'towerDeathEvents', 'radiantKills', 'lobbyType', 'replaySalt', 'buildingEvents', 'duration', 'id', 'direKills', 'gameVersionId', 'isStats', 'wardEvents', 'runeEvents'])

In [19]:
data['players'][0].keys()

dict_keys(['isRadiant', 'abilityActiveLists', 'profileUrl', 'leaverStatus', 'name', 'avatarMedium', 'towerDamage', 'steamId', 'heroDamage', 'partyRank', 'healEvents', 'deathEvents', 'inventoryEvent', 'playerUpdateBattleEvents', 'item1', 'abilityUsedEvents', 'item5', 'assistEvents', 'heroAvgImp', 'purchaseEvents', 'imp', 'goldSpent', 'numLastHits', 'item3', 'avatarFull', 'heroDamageEvents', 'item2', 'expPerMinute', 'numDeaths', 'killEvents', 'numDenies', 'buyBackEvents', 'experienceEvents', 'heroHealing', 'goldEvents', 'playerUpdateLevelEvents', 'goldPerMinute', 'soloRank', 'avatar', 'hero', 'csEvents', 'playerUpdateAttributeEvents', 'playerUpdateHealthEvents', 'role', 'numKills', 'playerUpdatePositionEvents', 'item4', 'level', 'playerUpdateGoldEvents', 'lane', 'numAssists', 'slot', 'item0', 'award', 'abilityLearnEvents', 'towerDamageEvents'])

In [20]:
print(data['players'][0]['role'])
print(data['players'][0]['lane'])
print(data['players'][0]['hero'])
print(data['players'][0]['purchaseEvents'])
for player in data['players']:
    print(player['slot'])

1
1
92
[{'time': -89, 'item': 188}, {'time': -89, 'item': 43}, {'time': -89, 'item': 29}, {'time': -89, 'item': 44}, {'time': -89, 'item': 38}, {'time': -85, 'item': 46}, {'time': -84, 'item': 46}, {'time': 187, 'item': 84}, {'time': 193, 'item': 42}, {'time': 236, 'item': 42}, {'time': 384, 'item': 42}, {'time': 389, 'item': 244}, {'time': 389, 'item': 27}, {'time': 432, 'item': 27}, {'time': 440, 'item': 214}, {'time': 509, 'item': 46}, {'time': 528, 'item': 42}, {'time': 603, 'item': 46}, {'time': 719, 'item': 46}, {'time': 720, 'item': 42}, {'time': 809, 'item': 25}, {'time': 811, 'item': 182}, {'time': 816, 'item': 42}, {'time': 882, 'item': 46}, {'time': 1059, 'item': 46}, {'time': 1264, 'item': 64}, {'time': 1264, 'item': 65}, {'time': 1277, 'item': 46}, {'time': 1533, 'item': 46}, {'time': 1758, 'item': 23}, {'time': 1758, 'item': 21}, {'time': 1758, 'item': 22}, {'time': 1760, 'item': 60}, {'time': 1760, 'item': 108}, {'time': 1919, 'item': 42}]
0
1
2
3
4
128
129
130
131
132


**Need to get steamIDs for players by going through the matchIDs of high level games**

In [21]:
def nextBatchOfMatches(idList):
    queryParams = {"matchId":idList, 
          "include": ["Player"],
         "gameMode": [2, 22],
         "lobbyType": [2, 7]}
    x = requests.get(host, params = queryParams)
    x = x.json()
    for entry in x['results']:
        for player in entry['players']:
            playerSet.add(player['steamId'])

In [22]:
if os.path.exists('playerset.txt'):
    playerSet = np.loadtxt('playerset.txt', dtype=np.int64)
else:
    playerSet = set()
    allMatches = np.loadtxt('matchids.txt', dtype=np.int64)
    allMatches = split(allMatches, 10)
    for matches in allMatches:
        nextBatchOfMatches(matches)
    playerSet = list(playerSet)
    np.savetxt('playerset.txt', playerSet, delimiter=',', fmt = "%.0f")

With player set, we can obtain matches. For the players above, get the matches these players play and also categorize the players and heroes

In [23]:
# 8 is jug and 74 is invoker and 106 is ember
def playerMatchObtain(player, heroList = [8, 74, 106]):
    host = "https://api.stratz.com/api/v1/match"
    queryParams = {"steamId": player, 
                   "heroId": heroList,
                   "gameMode": [2, 22],
                   "lobbyType": [2, 7],
                   "gameVersion": 79, 
                  "take": 100}
    x = requests.get(host, params = queryParams)
    x = x.json()
    for entry in x['results']:
        matchSet.add(entry['id'])

In [24]:
matchSet = set()
if os.path.exists('matchset.txt'):
    matchSet = np.loadtxt('matchset.txt', dtype=np.int64)
else:
    allplayers = np.loadtxt('playerset.txt', dtype=np.int64)
    for player in allplayers:
        playerMatchObtain(player, heroList=[74])
    matchSet = list(matchSet)
    np.savetxt('matchset.txt', matchSet, delimiter=',', fmt = "%.0f")

In [25]:
len(playerSet)

589

In [26]:
len(matchSet)

1542

** Time to Parse these Matches**
Need to make sure to remove tps and wards from buy list as you can buy them at any time depending on the situation

In [27]:
prototypeDataframe = {
    "invokerItems": [], "invokerSideRadiant": [], 
    "invokerSideHero1": [], "invokerSideHero2": [], 
    "invokerSideHero3": [], "invokerSideHero4": [],
    "otherSideHero1": [], "otherSideHero2": [],
    "otherSideHero3": [], "otherSideHero4": [],
    "otherSideHero5": [], 
    "invokerSideHero1Steam": [], "invokerSideHero2Steam": [], 
    "invokerSideHero3Steam": [], "invokerSideHero4Steam": [],
    "otherSideHero1Steam": [], "otherSideHero2Steam": [],
    "otherSideHero3Steam": [], "otherSideHero4Steam": [],
    "otherSideHero5Steam": [], "matchId": []
}
session = requests.Session()
def analyzeMatchesForFeatures(matchList, heroesToTestItems = [8, 74, 106], ):
    hostInitial = "https://api.stratz.com/api/v1/match/"
    rs = (grequests.get(hostInitial + str(match), session=session) for match in matchList)
    x = grequests.map(rs, size = 50)
    for matchUnparsed in x:
        if not matchUnparsed.ok:
            matchUnparsed.close()
            continue
        match = matchUnparsed.json()
        radiantHeroes = []
        direHeroes = []
        radiantSteams = []
        direSteams = []
        invokerSideRadiant = None
        for player in match['players']:
            # we have the invoker
            if player['hero'] == 74:
                invokerSideRadiant = player['slot'] < 5
                invokerItems = [event['item'] for event in player['purchaseEvents']]
            else:
                if player['slot'] < 5:
                    radiantHeroes.append(player['hero'])
                    radiantSteams.append(player['steamId'])
                else: 
                    direHeroes.append(player['hero'])
                    direSteams.append(player['steamId'])
        invokerWon = invokerSideRadiant == match['didRadiantWin']
        if not invokerWon:
            matchUnparsed.close()
            continue
        prototypeDataframe['matchId'].append(match['id'])
        prototypeDataframe['invokerSideRadiant'].append(invokerSideRadiant)
        prototypeDataframe['invokerItems'].append(invokerItems)
        if invokerSideRadiant:
            invokerSideHeroes = radiantHeroes
            invokerSideSteams = radiantSteams
            otherSideHeroes = direHeroes
            otherSideSteams = direSteams
        else:
            invokerSideHeroes = direHeroes
            invokerSideSteams = direSteams
            otherSideHeroes = radiantHeroes
            otherSideSteams = radiantSteams
        for ix1 in range(5):
            prototypeDataframe['otherSideHero' + str(ix1+1)].append(otherSideHeroes[ix1])
            prototypeDataframe['otherSideHero' + str(ix1+1) + 'Steam'].append(otherSideSteams[ix1])
            if ix1 != 4:
                prototypeDataframe['invokerSideHero' + str(ix1+1)].append(invokerSideHeroes[ix1])
                prototypeDataframe['invokerSideHero'+ str(ix1+1) + 'Steam'].append(invokerSideSteams[ix1])
        matchUnparsed.close()
            
def GetHeroHistory(steamid, heroid):
    host = "https://api.stratz.com/api/v1/match"
    queryParams = {"steamId": steamid, 
                   "heroId": [heroid],
                   "Player": "Single",
                   "gameMode": [2, 22],
                   "lobbyType": [2, 7],
                   "gameVersion": 79, 
                  "take": 100}
    x = requests.get(host, params = params)
    x = x.json()
    features = {'kills': [], 'deaths': [], 'lastHits': [], 'gold': [], 'obs': [], 'sens': []}
    for match in x['results']:
        pass

In [28]:
if not os.path.exists('invokerMatchFeatureSetPlus.csv'):
    analyzeMatchesForFeatures(matchSet)
    invokerMatchFeatureSetPlus = pd.DataFrame(prototypeDataframe)
    invokerMatchFeatureSetPlus.to_csv('invokerMatchFeatureSetPlus.csv', index=False)

[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [504]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [500]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response


KeyboardInterrupt

